In [1]:
from mitoolspro.regressions import panel_models
import pandas as pd
import statsmodels.api as sm
from linearmodels.datasets import wage_panel
from linearmodels.panel.data import PanelData
from linearmodels.panel import compare
from collections import OrderedDict

# Panel OLS from Linearmodels

https://bashtage.github.io/linearmodels/panel/examples/examples.html

These examples cover the models available for estimating panel models. The initial examples all ignore covariance options and so use the default classic covariance which is appropriate for homoskedastic data. The alternative covariance options are described at the end of this document.

## Loading data

These examples all make use of the wage panel from

6. Vella and M. Verbeek (1998), “Whose Wages Do Unions Raise? A Dynamic Model of Unionism and Wage Rate Determination for Young Men,” Journal of Applied Econometrics 13, 163-183.

The data set consists of wages and characteristics for men during the 1980s. The entity identifier is `nr` and the time identified is `year`. This data is used extensively in Chapter 14 of Introduction to Econometrics by Jeffrey Wooldridge.

Here a `MultiIndex DataFrame` is used to hold the data in a format that can be understood as a panel. Before setting the index, a year `Categorical` is created which facilitated making dummies.

```python

```

In [2]:
data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(["nr", "year"])
data["year"] = year
print(wage_panel.DESCR)
print(data.head())


F. Vella and M. Verbeek (1998), "Whose Wages Do Unions Raise? A Dynamic Model
of Unionism and Wage Rate Determination for Young Men," Journal of Applied
Econometrics 13, 163-183.

nr                       person identifier
year                     1980 to 1987
black                    =1 if black
exper                    labor market experience
hisp                     =1 if Hispanic
hours                    annual hours worked
married                  =1 if married
educ                     years of schooling
union                    =1 if in union
lwage                    log(wage)
expersq                  exper^2
occupation               Occupation code

         black  exper  hisp  hours  married  educ  union     lwage  expersq  \
nr year                                                                       
13 1980      0      1     0   2672        0    14      0  1.197540        1   
   1981      0      2     0   2320        0    14      1  1.853060        4   
   1982      0    

## Basic regression on panel data

`PooledOLS` is just plain OLS that understands that various panel data structures. It is useful as a base model. Here the log wage is modeled using all of the variables and time dummies.

In [3]:
exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
model = panel_models.PooledOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data)
pooled_res = model.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                  PooledOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.1893
Time:                        21:20:46   Log-likelihood                   -2982.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             72.459
                            

## Estimating parameters with uncorrelated effects

When modeling panel data it is common to consider models beyond what OLS will efficiently estimate. The most common are error component models which add an additional term to the standard OLS model, $$y_{it} = x_{it}\beta + \alpha_{i} + \epsilon_{it}$$ where $\alpha_{i}$ affects all values of entity i. When the $\alpha_{i}$ are uncorrelated with the regressors in $x_{it}$, a random effects model can be used to efficiently estimate parameters of this model.

### Random effects

The random effects model is virtually identical to the pooled OLS model except that is accounts for the structure of the model and so is more efficient. Random effects uses a quasi-demeaning strategy which subtracts the time average of the within entity values to account for the common shock.

In [4]:
model = panel_models.RandomEffectsModel(dependent_variable='lwage', independent_variables=exog_vars, data=data)
re_res = model.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.1828
Time:                        21:20:46   Log-likelihood                   -1622.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             68.409
                            

The model fit is fairly similar, although the return to experience has changed substantially, as has its significance. This is partially explainable by the inclusion of the year dummies which will fit the trend in experience and so only the cross-sectional differences matter. The quasi-differencing in the random effects estimator depends on a quantity that depends on the relative variance of the idiosyncratic shock and the common shock. This can be accessed using `variance_decomposition`.

In [5]:
re_res.variance_decomposition

Effects                   0.106946
Residual                  0.123324
Percent due to Effects    0.464438
Name: Variance Decomposition, dtype: float64

The coefficient $\theta_{i}$ determines how much demeaning takes place. When this value is 1, the RE model reduces to the pooled model since this occurs when there is no variance in the effects. When panels are unbalanced it will vary across entities, but in this balanced panel all values are the same.

In [6]:
re_res.theta.head()


,theta
nr,
13,0.645059
17,0.645059
18,0.645059
45,0.645059
110,0.645059


### The Between estimator

The between estimator is an alternative, usually less efficient estimator, can can be used to estimate model parameters. It is particular simple since it first computes the time averages of $y$ and $x$ and then runs a simple regression using these averages. 

The year dummies are dropped since the averaging removes differences due to the year. `expersq` was also dropped since it is fairly co-linear with `exper`. These results are broadly similar to the previous models.

In [7]:
exog_vars = ["black", "hisp", "exper", "married", "educ", "union"]
model = panel_models.BetweenOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data)
be_res = model.fit()
print(be_res)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.2155
Estimator:                 BetweenOLS   R-squared (Between):              0.2155
No. Observations:                 545   R-squared (Within):               0.1141
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.1686
Time:                        21:20:46   Log-likelihood                   -194.54
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.633
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                   F(6,538)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             24.633
                            

#### **Other options**

The `fit` method of `BetweenOLS` takes an optional input `reweight` which uses GLS-like weighting when panels are unbalanced. In this dataset the panel is balanced and so using this option would have no effect. By default this is `False` and so the averages are directly used in OLS even when there are difference numbers of observations across entities.



## Handling correlated effects

When effects are correlated with the regressors the RE and BE estimators are not consistent. The usual solution is to use Fixed Effects which are available in `PanelOLS`. Fixed effects are called `entity_effects` when applied to entities and `time_effects` when applied to the time dimension:

### Including fixed effects

Entity effects are included by setting `entity_effects=True`. This is equivalent to including dummies for each entity. In this panel, this would add 545 dummy variables and estimation of the model would be considerably slower. `PanelOLS` does not actually use dummy variables and instead uses group-wise demeaning to achieve the same effect.

#### **Time-invariant Variables**

Time-invariant variables cannot be included when using entity effects since, once demeaned, these will all be 0. Here `exper` is also excluded since once entity effects and time dummies are incorporated, `exper` is perfectly co-linear.

In [8]:
exog_vars = ["expersq", "union", "married", "year"]
model = panel_models.PanelOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data, entity_effects=True)
fe_res = model.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.0807
Time:                        21:20:46   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

### Time Effects

Time effect can be added using `time_effects=True`. Here the time dummies are removed. Note that the core coefficients are identical. The only change is in the test statistic for poolability since not the “effects” include both entity and time, whereas before only entity were included.


#### **Effects vs Dummies**

For variable which can be consistently estimated, such as time effects in the usual large N, fixed T panel, it is not necessary to include these as effects. They can instead be implemented as dummy variables.

In [9]:
exog_vars = ["expersq", "union", "married"]
model = panel_models.PanelOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data, time_effects=True, entity_effects=True)
fe_te_res = model.fit()
print(fe_te_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.0216
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):              -0.4809
Date:                Wed, Mar 19 2025   R-squared (Overall):             -0.2253
Time:                        21:20:46   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.959
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(3,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             27.959
                            

#### **Other options**

Some additional options are available when using `PanelOLS` and effects. The `use_lsdv` can be used to force the model to be estimated using dummy variables. This is not necessary and will be slower in most circumstances. This options is primarily included for testing. `auto_df` instructs `PanelOLS` to determine the degree of freedom adjustment to make. Unlike most other estimators, the treatment of effects depends on the covariance estimator. For example, when using a classic covariance estimator, effects count as lost degrees of freedom. When using entity effects and clustering by entity, they do not. Setting `auto_df=False` will allow the entities to be counted or not, depending on the value of `count_effects`.

### Other Effects

`PanelOLS` can be used to estimated models with up to 2-effects fixed effects. These can include any combination of

* Entity effects
* Time effects
* Other effects

Other effects are specified using the `other_effects` input of `PanelOLS`. The input should have the same number of observations as the engog and can have 1 or two columns. Below we reproduce the model fitted with time effects only using `other_effects` to set the effect.

In [10]:
exog_vars = ["expersq", "union", "married"]
exog = sm.add_constant(data[exog_vars])
time_ids = pd.DataFrame(
    PanelData(data.lwage).time_ids, index=data.index, columns=["Other Effect"]
)
time_ids

Other Effect
nr    year              
13    1980             0
      1981             1
      1982             2
      1983             3
      1984             4
...                  ...
12548 1983             3
      1984             4
      1985             5
      1986             6
      1987             7

[4360 rows x 1 columns]

In [11]:
model = panel_models.PanelOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data, entity_effects=True, other_effects=time_ids)
fe_oe_res = model.fit()
print(fe_oe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.0216
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):              -0.4809
Date:                Wed, Mar 19 2025   R-squared (Overall):             -0.2253
Time:                        21:20:46   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.959
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(3,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             27.959
                            

### Using first differences

First differencing is an alternative to using fixed effects when there might be correlation. When using first differences, time-invariant variables must be excluded. Additionally, only one linear time-trending variable can be included since this will look like a constant. This variable will soak up all time-trends in the data, and so interpretations of these variable can be challenging.

In [12]:
exog_vars = ["exper", "expersq", "union", "married"]
model = panel_models.FirstDifferenceOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data)
fd_res = model.fit()
print(fd_res)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  lwage   R-squared:                        0.0268
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.5491
No. Observations:                3815   R-squared (Within):               0.1763
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.5328
Time:                        21:20:46   Log-likelihood                   -2305.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      26.208
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(4,3811)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             26.208
                            

## Comparing models

Model results can be compared using `compare`. `compare` accepts lists of results, a dictionary of results where the key is interpreted as the model name.

In [13]:
print(compare({"BE": be_res, "RE": re_res, "Pooled": pooled_res}))

                        Model Comparison                       
                                BE              RE       Pooled
---------------------------------------------------------------
Dep. Variable                lwage           lwage        lwage
Estimator               BetweenOLS   RandomEffects    PooledOLS
No. Observations               545            4360         4360
Cov. Est.               Unadjusted      Unadjusted   Unadjusted
R-squared                   0.2155          0.1806       0.1893
R-Squared (Within)          0.1141          0.1799       0.1692
R-Squared (Between)         0.2155          0.1853       0.2066
R-Squared (Overall)         0.1686          0.1828       0.1893
F-statistic                 24.633          68.409       72.459
P-value (F-stat)            0.0000          0.0000       0.0000
===================== ============ =============== ============
const                       0.2836          0.0234       0.0921
                          (1.5897)      

## Covariance options

### Heteroskedasticity Robust Covariance

White”s robust covariance can be used by setting `cov_type="robust"`. This estimator adds some robustness against certain types of specification issues but should not be used when using fixed effects (entity effects) since it is no longer robust. Instead a clustered covariance is required.

In [14]:
exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union"]
model = panel_models.PooledOLSModel(dependent_variable='lwage', independent_variables=exog_vars, data=data)
robust = model.fit(cov_type="robust")

### Clustered by Entity

The usual variable to cluster are are entity or entity and time. The can be implemented using `cov_type="clustered"` and the additional keyword arguments `cluster_entity=True` and/or `cluster_time=True`.

In [15]:
clust_entity = model.fit(cov_type="clustered", cluster_entity=True)

This next example clusters by both.

In [16]:
clust_entity_time = model.fit(
    cov_type="clustered", cluster_entity=True, cluster_time=True
)

An `OrderedDict` is used to hold the results for comparing models. This allows the models to be named as well as for the order of the models to be specified. A standard `dict` will produce effectively random order.

Clustering on entity reduced the t-stats across the board. This suggests there is important correlation in the residuals per entity. Clustering by both also decreases the t-stats which suggests that there is cross-sectional dependence in the data. *Note:* clustering by entity addresses correlation across time and clustering by time controls for correlation between entities in a time period.

In [17]:
res = OrderedDict()
res["Robust"] = robust
res["Entity"] = clust_entity
res["Entity-Time"] = clust_entity_time
print(compare(res))

                     Model Comparison                    
                           Robust      Entity Entity-Time
---------------------------------------------------------
Dep. Variable               lwage       lwage       lwage
Estimator               PooledOLS   PooledOLS   PooledOLS
No. Observations             4360        4360        4360
Cov. Est.                  Robust   Clustered   Clustered
R-squared                  0.1866      0.1866      0.1866
R-Squared (Within)         0.1679      0.1679      0.1679
R-Squared (Between)        0.2027      0.2027      0.2027
R-Squared (Overall)        0.1866      0.1866      0.1866
F-statistic                142.61      142.61      142.61
P-value (F-stat)           0.0000      0.0000      0.0000
===================== =========== =========== ===========
const                     -0.0347     -0.0347     -0.0347
                        (-0.5360)   (-0.2892)   (-0.3145)
black                     -0.1438     -0.1438     -0.1438
              

### Other clusters

Other clusters can be used by directly passing integer arrays (1 or 2 columns, or a 1-d array) using the input `clusters`. This example clustered by occupation, which is probably not a reliable variable to cluster on since there are only 9 groups and the usual theory for clustered standard errors requires that the number of clusters is large.

In [18]:
clust_entity = model.fit(cov_type="clustered", clusters=data.occupation)
print(data.occupation.value_counts())
print(clust_entity)

occupation
5    934
6    881
9    509
4    486
1    453
7    401
2    399
3    233
8     64
Name: count, dtype: int64
                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1866
Estimator:                  PooledOLS   R-squared (Between):              0.2027
No. Observations:                4360   R-squared (Within):               0.1679
Date:                Wed, Mar 19 2025   R-squared (Overall):              0.1866
Time:                        21:20:47   Log-likelihood                   -2989.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      142.61
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(7,4352)
Min Obs:                       8.0000                                   

***